In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset


class MetricsEvaluationVisualization:
    """
    PyTorch深度学习模型性能指标评估与可视化模块
    用于输出模型指标（Accuracy、Precision、Recall、F1-score），
    并绘制混淆矩阵与特征分布图，强化模型理解与诊断能力。
    """

    def __init__(self,
                 model,
                 test_loader: DataLoader,
                 device=None,
                 class_names=None,
                 loss_fn=None,
                 train_history=None,
                 valid_history=None):
        """
        :param model: 训练好的机器学习模型
        :param test_loader: 测试数据加载器
        :param device: 'cpu'或'cuda'
        :param class_names: 类别名称列表（可选）
        :param loss_fn: 损失函数（可选，用于计算测试损失）
        :param train_history: 训练过程中的指标记录（可选，用于绘制训练曲线）
        :param valid_history: 验证过程中的指标记录（可选，用于绘制训练曲线）
        """
        if device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = device
        self.model = model.to(self.device)
        self._check_mode()
        self.test_loader = test_loader
        self.loss_fn = loss_fn
        self.categories = class_names
        self.train_history = train_history
        self.valid_history = valid_history

        # 用于存储评估结果
        self.y_true = []
        self.y_pred = []
        self.y_score = []
        self.test_loss: float = 0.0

    def _check_mode(self):
        """确保模型处于评估模式"""
        if self.model.training:
            print("模型处于训练模式，切换到评估模式")
            self.model.eval()
        else:
            print("模型已处于评估模式")
            self.model.eval()

    def evaluate(self, return_outputs=False):
        """
        评估模型性能指标
        :param return_outputs: 是否返回所有预测的结果
        :return:evaluation_metrics: 评估指标字典
        :return:(可选) outputs: 包含真实标签、预测标签和预测概率的元组
        """
